__Note__: The code used here was heavily inspired by the AI for Medicine Specialization Course 1 Week three programming assignments: [link](https://www.coursera.org/learn/ai-for-medical-diagnosis)

In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import json
import os
from pprint import pprint
from collections import OrderedDict
from ipywidgets import interact, interactive, IntSlider, ToggleButtons
from sys import getsizeof
from tqdm import tqdm

### Reading dataset description

In [2]:
file = open('../datasets_descriptions_json/task01_braintumor_dataset.json')
data_description = json.load(file)

In [3]:
print(json.dumps(data_description, indent=4))

{
    "name": "BRATS",
    "description": "Gliomas segmentation tumour and oedema in on brain images",
    "reference": "https://www.med.upenn.edu/sbia/brats2017.html",
    "licence": "CC-BY-SA 4.0",
    "release": "2.0 04/05/2018",
    "tensorImageSize": "4D",
    "modality": {
        "0": "FLAIR",
        "1": "T1w",
        "2": "t1gd",
        "3": "T2w"
    },
    "labels": {
        "0": "background",
        "1": "edema",
        "2": "non-enhancing tumor",
        "3": "enhancing tumour"
    },
    "numTraining": 484,
    "numTest": 266,
    "training": [
        {
            "image": "./imagesTr/BRATS_457.nii.gz",
            "label": "./labelsTr/BRATS_457.nii.gz"
        },
        {
            "image": "./imagesTr/BRATS_306.nii.gz",
            "label": "./labelsTr/BRATS_306.nii.gz"
        },
        {
            "image": "./imagesTr/BRATS_206.nii.gz",
            "label": "./labelsTr/BRATS_206.nii.gz"
        },
        {
            "image": "./imagesTr/BRATS_449.nii.

### Reading and visualizing BRATS_001 nifti MRI and mask files

#### MRI file

In [4]:
mri_1 = nib.load("../../datasets/Task01_BrainTumour/train/images/BRATS_001.nii.gz")

In [5]:
type(mri_1)

nibabel.nifti1.Nifti1Image

In [6]:
# Read Nifi Image object as numpy file
mri_1 = mri_1.get_fdata()

In [7]:
type(mri_1)

numpy.ndarray

In [8]:
mri_1.dtype

dtype('float64')

In [9]:
mri_1.shape

(240, 240, 155, 4)

In [10]:
# Range of voxel values (maximum value - minimum value)
print(mri_1[:, :, :, 0].ptp()) # Channel 0
print(mri_1[:, :, :, 1].ptp()) # Channel 1
print(mri_1[:, :, :, 2].ptp()) # Channel 2
print(mri_1[:, :, :, 3].ptp()) # Channel 3

1829.0
2155.0
2239.0
1669.0


In [11]:
getsizeof(mri_1) # 285.696144 MBs

285696144

In [12]:
getsizeof(mri_1.astype(np.float32)) # 142.848144 MBs

142848144

In [13]:
getsizeof(mri_1.astype(np.float16)) # 71.424144 MBs

71424144

#### Mask file

In [14]:
mri_1_mask = nib.load("../../datasets/Task01_BrainTumour/train/masks/BRATS_001.nii.gz")

In [15]:
mri_1_mask = mri_1_mask.get_fdata()

In [16]:
mri_1_mask.dtype

dtype('float64')

In [17]:
np.unique(mri_1_mask)

array([0., 1., 2., 3.])

In [18]:
getsizeof(mri_1_mask) # 71.424128 MBs

71424128

In [19]:
getsizeof(mri_1_mask.astype(np.float32)) # 35.712128 MBs

35712128

In [20]:
getsizeof(mri_1_mask.astype(np.uint8)) # 8.928128 MBs

8928128

In [21]:
mri_1_mask = mri_1_mask.astype(np.float32)

In [22]:
mri_1_mask.shape

(240, 240, 155)

### Visualizing nifti MRI file and Mask file

In [23]:
classes_dict = {
    'Background': 0,
    'Edema': 1,
    'Non-enhancing tumor': 2,
    'Enhancing tumor': 3 
}

# Create button values
select_class = ToggleButtons(
    options=['Background','Edema', 'Non-enhancing tumor', 'Enhancing tumor', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=154, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer, channel):
    print(f"Plotting Layer: {layer} | Label: {seg_class} | Channel: {channel}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 2, 1)
    plt.imshow(mri_1[:, :, layer, channel], cmap='gray');
    plt.title("MRI", fontsize=20)
    plt.axis('off')
    
    fig.add_subplot(1, 2, 2)
    if seg_class == "All":
        mask = mri_1_mask[:, :, layer]
        plt.title("Mask", fontsize=20)
        plt.imshow(mask)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask = np.where(mri_1_mask[:, :, layer] == img_label, 255, 0)
        plt.title("Mask", fontsize=20)
        plt.imshow(mask, cmap='gray')
        plt.axis('off');

# Set channel to view:
#  Channel 0: "FLAIR" Fluid-attenuated inversion recovery
#  Channel 1: "T1w" T1-weighted
#  Channel 2: "t1gd" T1-weighted with gadolinium contrast enhancement
#  Channel 3: "T2w" T2-weighted    
    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer, channel=(0, 3))

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…

### Reading and visualizing BRATS_002 nifti MRI and mask files

#### MRI file

In [24]:
mri_2 = nib.load("../../datasets/Task01_BrainTumour/train/images/BRATS_002.nii.gz")

In [25]:
type(mri_2)

nibabel.nifti1.Nifti1Image

In [26]:
# Read Nifi Image object as numpy file
mri_2 = mri_2.get_fdata()

In [27]:
type(mri_2)

numpy.ndarray

In [28]:
mri_2.dtype

dtype('float64')

In [29]:
mri_2.shape

(240, 240, 155, 4)

In [30]:
# Range of voxel values (maximum value - minimum value)
print(mri_2[:, :, :, 0].ptp()) # Channel 0
print(mri_2[:, :, :, 1].ptp()) # Channel 1
print(mri_2[:, :, :, 2].ptp()) # Channel 2
print(mri_2[:, :, :, 3].ptp()) # Channel 3

1905.0
2869.0
2656.0
1605.0


#### Mask file

In [31]:
mri_2_mask = nib.load("../../datasets/Task01_BrainTumour/train/masks/BRATS_002.nii.gz")

In [32]:
mri_2_mask = mri_2_mask.get_fdata()

In [33]:
mri_2_mask.dtype

dtype('float64')

In [34]:
mri_2_mask = mri_2_mask.astype(np.uint8)

In [35]:
mri_2_mask.shape

(240, 240, 155)

### Visualizing nifti MRI file and Mask file

In [36]:
classes_dict = {
    'Background': 0,
    'Edema': 1,
    'Non-enhancing tumor': 2,
    'Enhancing tumor': 3 
}

# Create button values
select_class = ToggleButtons(
    options=['Background','Edema', 'Non-enhancing tumor', 'Enhancing tumor', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=154, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer, channel):
    print(f"Plotting Layer: {layer} | Label: {seg_class} | Channel: {channel}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 2, 1)
    plt.title("MRI", fontsize=20)
    plt.imshow(mri_2[:, :, layer, channel], cmap='gray');
    plt.axis('off')
    
    fig.add_subplot(1, 2, 2)
    if seg_class == "All":
        mask = mri_2_mask[:, :, layer]
        plt.title("Mask", fontsize=20)
        plt.imshow(mask)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask = np.where(mri_2_mask[:, :, layer] == img_label, 255, 0)
        plt.title("Mask", fontsize=20)
        plt.imshow(mask, cmap='gray')
        plt.axis('off');

# Set channel to view:
#  Channel 0: "FLAIR" Fluid-attenuated inversion recovery
#  Channel 1: "T1w" T1-weighted
#  Channel 2: "t1gd" T1-weighted with gadolinium contrast enhancement
#  Channel 3: "T2w" T2-weighted    
    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer, channel=(0, 3))

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…

### Reading and visualizing BRATS_003 nifti MRI and mask files

#### MRI file

In [37]:
mri_3 = nib.load("../../datasets/Task01_BrainTumour/train/images/BRATS_003.nii.gz")

In [38]:
type(mri_3)

nibabel.nifti1.Nifti1Image

In [39]:
# Read Nifi Image object as numpy file
mri_3 = mri_3.get_fdata()

In [40]:
type(mri_3)

numpy.ndarray

In [41]:
mri_3.dtype

dtype('float64')

In [42]:
mri_3.shape

(240, 240, 155, 4)

In [43]:
# Range of voxel values (maximum value - minimum value)
print(mri_1[:, :, :, 0].ptp()) # Channel 0
print(mri_1[:, :, :, 1].ptp()) # Channel 1
print(mri_1[:, :, :, 2].ptp()) # Channel 2
print(mri_1[:, :, :, 3].ptp()) # Channel 3

1829.0
2155.0
2239.0
1669.0


#### Mask file

In [44]:
mri_3_mask = nib.load("../../datasets/Task01_BrainTumour/train/masks/BRATS_003.nii.gz")

In [45]:
mri_3_mask = mri_3_mask.get_fdata()

In [46]:
mri_3_mask.dtype

dtype('float64')

In [47]:
mri_3_mask = mri_3_mask.astype(np.uint8)

In [48]:
mri_3_mask.shape

(240, 240, 155)

### Visualizing nifti MRI file and Mask file

In [49]:
classes_dict = {
    'Background': 0,
    'Edema': 1,
    'Non-enhancing tumor': 2,
    'Enhancing tumor': 3 
}

# Create button values
select_class = ToggleButtons(
    options=['Background','Edema', 'Non-enhancing tumor', 'Enhancing tumor', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=154, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer, channel):
    print(f"Plotting Layer: {layer} | Label: {seg_class} | Channel: {channel}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 2, 1)
    plt.title("MRI", fontsize=20)
    plt.imshow(mri_3[:, :, layer, channel], cmap='gray');
    plt.axis('off')
    
    fig.add_subplot(1, 2, 2)
    if seg_class == "All":
        mask = mri_3_mask[:, :, layer]
        plt.title("Mask", fontsize=20)
        plt.imshow(mask)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask = np.where(mri_3_mask[:, :, layer] == img_label, 255, 0)
        plt.title("Mask", fontsize=20)
        plt.imshow(mask, cmap='gray')
        plt.axis('off');

# Set channel to view:
#  Channel 0: "FLAIR" Fluid-attenuated inversion recovery
#  Channel 1: "T1w" T1-weighted
#  Channel 2: "t1gd" T1-weighted with gadolinium contrast enhancement
#  Channel 3: "T2w" T2-weighted    
    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer, channel=(0, 3))

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…

### Calculating MRI shape counter dictionary

In [50]:
data_dir = "../../datasets/Task01_BrainTumour/"

train_mri_paths = [os.path.join(data_dir + "train/images", x) for x in os.listdir(data_dir + "train/images")]
val_mri_paths = [os.path.join(data_dir + "val/images", x) for x in os.listdir(data_dir + "val/images")]
test_mri_paths = [os.path.join(data_dir + "test_images_for_model_prediction_submission", x) for x in os.listdir(data_dir + "test_images_for_model_prediction_submission")]

all_paths = train_mri_paths + val_mri_paths + test_mri_paths  # concatenate list of paths

shape_count_dict = {}

for path in tqdm(all_paths):
    mri = nib.load(path).get_fdata()
    shape = mri.shape
    
    if shape not in shape_count_dict.keys():
        shape_count_dict[shape] = 1
    else:
        shape_count_dict[shape] += 1   

100%|██████████| 750/750 [05:00<00:00,  2.50it/s]


In [51]:
shape_count_dict = OrderedDict(sorted(shape_count_dict.items(), key=lambda item: item[1], reverse=True))
pprint(shape_count_dict)

OrderedDict([((240, 240, 155, 4), 750)])


In [52]:
list_layers = []
for key, val in shape_count_dict.items():
    list_layers.append(key[2])

layers = np.array(list_layers)
print(layers.mean())
print(np.median(layers))
print(layers.min())
print(layers.max())

155.0
155.0
155
155
